In [1]:
%matplotlib inline

import numpy as np
from collections import OrderedDict, Counter, defaultdict
import pandas as pd

import Bio
from Bio import SeqIO

import seaborn as sns
import matplotlib.pyplot as plt

import glob

import subprocess
from subprocess import call

import re

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas

import pickle

from scipy import stats

import numpy as np
from scipy.spatial.distance import cdist


Bad key text.latex.unicode in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file /home/alla/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https://githu

In [2]:
# open SET1 (PhyloSET) and SET2 (RiboSET)
PhyloSET = pd.read_csv('../../tmp_res/SET1.txt', sep='\t')



RiboSET = pd.read_csv('../../tmp_res/SET2.txt', sep='\t')

RiboSET['tr_id1'] = [x.split('.')[0] for x in RiboSET['tr_id'].tolist()]

RiboSET = RiboSET[(~RiboSET['Riboseq_Summary'].str.contains('ATG')) & 
                  (~RiboSET['tr_id'].str.contains('PAR'))]

print (RiboSET.shape[0])

390


In [3]:
metadata_pc_g25 = pd.read_csv('../../tmp_res/metadata_pc_g25.txt', sep='\t')
metadata_pc_g25[0:1]

,tr_id,gene,gene_tr,transcript_seq,5UTR_start_seq,CDS_start,cds_seq,cds_start_codon,cds_stop_codon,cds_start_pos,cds_stop_pos,utr5_start,utr5_end,record_id
0,ENST00000335137.3,OR4F5,OR4F5-001,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,NaN,ATG,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,ATG,TAG,0,918,0,0,ENST00000335137.3|ENSG00000186092.4|OTTHUMG000...


In [4]:
# open metadata with scores and metrics
meta = pd.read_csv('../../tmp_res/local_and_global_df_g25ovlp_g35ovlp_g38ovlp_refseqovlp_strand_FRAMES_PhyloCSF.txt', sep='\t')
meta[0:1]

,tr_id,N_term_start,N_term_end,len,N_term_start1,N_term_end1,len_codons,gene,50len_flag,global_coo_50_and_less,...,max_sum_overlap_refseq_all_exons,max_sum_overlap_refseq_all_exons_strandless,ovlp,ovlp_strandless,ovlp2,ovlp2_strandless,PhyloCSF120score,tr_id1,number_of_records,records_names
0,ENST00000624431.1,274,316,42,274,316,14.0,FO538757.2,274,chr1:182667-182708,...,0,0,0;0;0,0;0;0,0;0;0;0,0;0;0;0,-163.9926,ENST00000624431,85,Human; Gorilla; Orangutan; Golden_snub_nosed_m...


# Trips-viz pred genes: exclude them 

In [5]:
transl_with_all_codons = pd.read_csv('../../data/extensions_TripsViz_1perGene_AUG_NearCognates.txt', sep=',')
transl_with_all_codons.columns = ['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 'len_ext', 
                                 'global_rank_ext', 'type_all_codons', 'trips_viz_link', 'Start_codon_ext', 
                                 'Highframe rank', 'Highframe value',
       'Lowframe rank', 'Lowframe value', 'Stop rank', 'Stop value',
       'Start rank', 'Start value', 'Coverage rank', 'Coverage_value_ext',
       'Amino acid sequence', 'Proteomics_count_ext', 'CDS_ratio']
transl_with_all_codons2 = transl_with_all_codons[['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 
                                                 'global_rank_ext', 'Start_codon_ext', 
                                                 'Proteomics_count_ext', 'Coverage_value_ext', 'CDS_ratio']]
transl_with_all_codons2['Riboseq_Summary'] = transl_with_all_codons2['Start_codon_ext']+'; '+transl_with_all_codons2['Start_ext'].astype(str)+'-'+transl_with_all_codons2['Stop_ext'].astype(str)+'; Rank: '+transl_with_all_codons2['global_rank_ext'].astype(str)+'; cov: '+np.round(transl_with_all_codons2['Coverage_value_ext'],2).astype(str)

top5000 = transl_with_all_codons2[0:5000]

top5000_meta = top5000.merge(meta, on='tr_id1', how='inner')

top5000_meta['len_codons'] = (top5000_meta['Stop_ext'] - top5000_meta['Start_ext']) // 3

# theoretical extensions should be no less than 20 codons (works for initiatial filter and for PhyloSET)
# but in practice it could be shorter
# let's say 10aa

# select 0 ovlps and where theoretical extension is at least 20 codons  
RiboSET_EXT = top5000_meta[(top5000_meta['ovlp'] == '0;0;0') & 
                                  (top5000_meta['Start_codon_ext'] != 'ATG') & 
                                  (top5000_meta['tr_id1'].isin(meta[meta['len_codons'] >= 20]['tr_id1'].tolist()))]

print (RiboSET_EXT.shape[0])

# ~4k RiboSET ext with non-zero CDS cov in my data 
RiboSET_EXT_cds_cov_nonzero = pd.read_csv('../tmp_res/RiboSET_EXT_cds_cov_nonzero_theor_len_20codons_3663genes.txt', sep='\t')
RiboSET_EXT_cds_cov_nonzero = RiboSET_EXT_cds_cov_nonzero.merge(RiboSET_EXT, on='tr_id1', how='inner')
RiboSET_EXT_cds_cov_nonzero = RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['NTE_coverage'] > 0]
RiboSET_EXT_cds_cov_nonzero = RiboSET_EXT_cds_cov_nonzero[~RiboSET_EXT_cds_cov_nonzero['tr_id'].str.contains('PAR')]
RiboSET_EXT_cds_cov_nonzero.to_csv('../tmp_res/FINAL_RiboSET_EXT_cds_cov_nonzero_theor_len_20codons_3532genes.txt', 
                                  sep='\t', index=False)

print (RiboSET_EXT_cds_cov_nonzero.shape[0])

RiboSET_EXT_cds_cov_nonzero = RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['CDS_coverage'] > 0.06]

RiboSET_EXT_cds_cov_nonzero.to_csv('../tmp_res/FINAL_RiboSET_EXT_cds_cov_nonzero_theor_len_20codons_3532genes.txt', 
                                  sep='\t', index=False)

print ('final shape of RiboSET_EXT', RiboSET_EXT_cds_cov_nonzero.shape[0], 'min CDS cov:', RiboSET_EXT_cds_cov_nonzero['CDS_coverage'].min())

print (len(set(RiboSET_EXT_cds_cov_nonzero['gene'].tolist()).intersection(RiboSET['gene'].tolist())))

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


3722
3530
final shape of RiboSET_EXT 3451 min CDS cov: 0.060654429369513166
390


In [6]:
RiboSET_EXT_cds_cov_nonzero[RiboSET_EXT_cds_cov_nonzero['Coverage_value_ext'] == 0][['NTE_nonzero_fraction', 'global_rank_ext']]

,NTE_nonzero_fraction,global_rank_ext
1335,0.200000,1786
1506,0.319444,2030
2075,0.307018,2803
2078,0.120000,2807
2126,0.238095,2871
...,...,...
3621,0.031746,4904
3623,0.015152,4910
3640,0.009009,4942
3646,0.009524,4956


In [7]:
transl_with_all_codons = pd.read_csv('../../data/extensions_TripsViz_1perGene_AUG_NearCognates.txt', sep=',')

transl_with_all_codons.columns = ['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 'len_ext', 
                                 'global_rank_ext', 'type_all_codons', 'trips_viz_link', 'Start_codon_ext', 
                                 'Highframe rank', 'Highframe value',
       'Lowframe rank', 'Lowframe value', 'Stop rank', 'Stop value',
       'Start rank', 'Start value', 'Coverage rank', 'Coverage_value_ext',
       'Amino acid sequence', 'Proteomics_count_ext', 'CDS_ratio']

transl_with_all_codons2 = transl_with_all_codons[['gene_ext', 'tr_id1', 'Start_ext', 'Stop_ext', 
                                                 'global_rank_ext', 'Start_codon_ext', 
                                                 'Proteomics_count_ext', 'Coverage_value_ext', 'CDS_ratio']]

transl_with_all_codons2['Riboseq_Summary'] = transl_with_all_codons2['Start_codon_ext']+'; '+transl_with_all_codons2['Start_ext'].astype(str)+'-'+transl_with_all_codons2['Stop_ext'].astype(str)+'; Rank: '+transl_with_all_codons2['global_rank_ext'].astype(str)+'; cov: '+np.round(transl_with_all_codons2['Coverage_value_ext'],2).astype(str)


transl_with_all_codons2[0:5]

/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,gene_ext,tr_id1,Start_ext,Stop_ext,global_rank_ext,Start_codon_ext,Proteomics_count_ext,Coverage_value_ext,CDS_ratio,Riboseq_Summary
0,ASPH,ENST00000379454,144,189,1,CTG,0.0,100.000000,1.645069,CTG; 144-189; Rank: 1; cov: 100.0
1,HSP90AA1,ENST00000216281,168,207,2,GTG,0.0,100.000000,3.081078,GTG; 168-207; Rank: 2; cov: 100.0
2,CD24,ENST00000619133,408,441,3,CTG,0.0,83.333333,11.578498,CTG; 408-441; Rank: 3; cov: 83.33
3,HN1,ENST00000409753,242,287,4,TTG,0.0,100.000000,0.588389,TTG; 242-287; Rank: 4; cov: 100.0
4,YBX1,ENST00000321358,44,140,5,CTG,0.0,85.185185,7.709763,CTG; 44-140; Rank: 5; cov: 85.19


In [8]:
transl_with_all_codons[transl_with_all_codons['Coverage_value_ext'] <= 5].sort_values(by='Coverage_value_ext')

,gene_ext,tr_id1,Start_ext,Stop_ext,len_ext,global_rank_ext,type_all_codons,trips_viz_link,Start_codon_ext,Highframe rank,...,Lowframe value,Stop rank,Stop value,Start rank,Start value,Coverage rank,Coverage_value_ext,Amino acid sequence,Proteomics_count_ext,CDS_ratio
1785,TPM4,ENST00000344824,59,119,61,1786,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",CTG,749,...,-10,437,-240,212,1356,587,0.0,LPQPPQSPPRTPRPPRQRP,0.000000,0.041187
8498,DNAJC7,ENST00000457167,-1,0,0,8499,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",NaN,488,...,-1,211,-1,857,-1,587,0.0,NaN,0.000000,0.000000
8497,UBXN4,ENST00000272638,-1,0,0,8498,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",NaN,488,...,-1,211,-1,857,-1,587,0.0,NaN,0.000000,0.000000
8496,SUN2,ENST00000405510,-1,0,0,8497,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",NaN,488,...,-1,211,-1,857,-1,587,0.0,NaN,0.000000,0.000000
8495,SLC36A1,ENST00000243389,-1,0,0,8496,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",NaN,488,...,-1,211,-1,857,-1,587,0.0,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4546,RAB40A,ENST00000372633,1985,2120,136,4547,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",ATT,487,...,1,210,0,855,1,538,5.0,ISLYFTDFPISFFMFYANFSRRTGPAPPLRTTPRAWLRRECGAS,0.000000,0.000000
4626,ST8SIA3,ENST00000324000,1960,2035,76,4627,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",ATA,495,...,-1,210,0,855,1,538,5.0,IYTPASCWPLNGPISPVSLNPAQP,0.000000,0.000000
4536,NECAB1,ENST00000417640,263,338,76,4537,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",CTG,487,...,0,210,0,854,2,538,5.0,LRPRPCQSRCVRLAPLRLRPSGLP,0.000000,0.000000
4557,BHMG1,ENST00000457052,342,417,76,4558,extension,"<a href=""https://trips.ucc.ie/homo_sapiens/Gen...",ATC,487,...,0,210,0,856,0,538,5.0,IGLPRVGPGETRTAGAASRHFARP,0.684211,0.000000


# Genes with no translation upstream in theor.ext but with CDS translation 

In [9]:
# get total list of transcripts 

amb_path = '../../data/counts_ext/download/N_term_amb/*'
unamb_path = '../../data/counts_ext/download/N_term_unamb/*'

list_of_tr = list(set([tr_path.split('/')[-1].split('_counts')[0] for tr_path in glob.glob(unamb_path)+glob.glob(amb_path)]))

list_of_amb_tr = [tr_path.split('/')[-1].split('_counts')[0] for tr_path in glob.glob(amb_path)]

list_of_unamb_tr = [tr_path.split('/')[-1].split('_counts')[0] for tr_path in glob.glob(unamb_path)]

print (list_of_tr[0:3])

print (len(list_of_tr), len(list_of_amb_tr), len(list_of_unamb_tr))

['ENST00000403106', 'ENST00000381916', 'ENST00000423831']
76910 75232 75139


In [10]:
transl_with_all_codons2['gene_ext'].tolist()

['ASPH',
 'HSP90AA1',
 'CD24',
 'HN1',
 'YBX1',
 'HNRNPA0',
 'NDUFA2',
 'GPRC5A',
 'SFPQ',
 'NPM1',
 'TK1',
 'CCNB1',
 'CD59',
 'LRRC59',
 'ATP5E',
 'YWHAE',
 'PRC1',
 'NONO',
 'MRFAP1',
 'HDGF',
 'NSUN2',
 'STMN1',
 'EZR',
 'HM13',
 'JUN',
 'LY6K',
 'NMU',
 'FTL',
 'NDUFS5',
 'LGALS1',
 'ID2',
 'COPG1',
 'ERVK3-1',
 'DDIT4',
 'DUSP5',
 'COX6C',
 'KDM1A',
 'FSTL1',
 'PSMD13',
 'YBX3',
 'PLEKHB2',
 'COX7C',
 'LARS',
 'GSPT1',
 'RAD1',
 'PPP1R14B',
 'ST13',
 'LAMP2',
 'PON2',
 'SLC25A1',
 'RBBP4',
 'PLIN3',
 'SYNCRIP',
 'RPA1',
 'KHDRBS1',
 'LASP1',
 'ASF1B',
 'H1FX',
 'F3',
 'CD151',
 'TGFB1',
 'UCK2',
 'LAPTM4B',
 'SLC25A32',
 'CDK2',
 'NDUFB10',
 'RRM1',
 'EPRS',
 'C1QBP',
 'MAPKAPK2',
 'TCEAL9',
 'FADD',
 'PPP1CB',
 'AARS',
 'SYAP1',
 'C9ORF16',
 'EREG',
 'SUPT4H1',
 'HMGCS1',
 'PNPLA2',
 'GALE',
 'LDB1',
 'BLMH',
 'PDCL',
 'HMGN1',
 'PMAIP1',
 'NELFB',
 'DESI1',
 'VMP1',
 'RCN1',
 'SUPT16H',
 'LIPA',
 'DYNLL2',
 'PFKM',
 'BAG6',
 'SPCS1',
 'PEG10',
 'EIF2B1',
 'SNRNP25',
 'SLC39A14'

In [11]:
# choose transcripts with no rank at al  

SEL_TR = list(set(list_of_tr) - set(transl_with_all_codons2[0:7000]['tr_id1'].tolist()))

print (len(SEL_TR), len(list_of_tr))
print (SEL_TR[0:2])

69997 76910
['ENST00000403106', 'ENST00000381916']


In [16]:
metadata_pc_g25['tr_len'] = [len(x) for x in metadata_pc_g25['transcript_seq'].tolist()]

full_cds = [x.split('.')[0] for x in metadata_pc_g25[(metadata_pc_g25['cds_stop_pos'] <= metadata_pc_g25['tr_len'])].tr_id.tolist()]

print ('full CDS len:', len(full_cds))

# length of theor ext is >=20 codons!!! 
SEL_TR_full_CDS = list(set(full_cds).intersection(SEL_TR).intersection(meta[(meta['len_codons'] >= 19)].tr_id1.tolist()))

len(SEL_TR_full_CDS)

full CDS len: 94359


33089

In [17]:
a = pd.DataFrame(set(full_cds).intersection(meta[(meta['len_codons'] >= 19)].tr_id1.tolist()))
a.columns = ['tr_id1']
a.merge(meta[['tr_id1', 'gene']]).gene.nunique(), a.merge(meta[['tr_id1', 'gene']]).tr_id1.nunique()

(12934, 39121)

In [22]:
b = pd.DataFrame(set(full_cds).intersection(meta[(meta['len_codons'] >= 20)].tr_id1.tolist()) - set(transl_with_all_codons2[0:5000]['tr_id1'].tolist()))
b.columns = ['tr_id1']
b = b.merge(meta[['tr_id1', 'gene']])

# exclude genes 
b = b[~b['gene'].isin(transl_with_all_codons2['gene_ext'].tolist())]

b.gene.nunique(), b.tr_id1.nunique()

(3523, 8577)

In [24]:
b = pd.DataFrame(set(full_cds).intersection(meta[(meta['len_codons'] >= 20)].tr_id1.tolist()) - set(transl_with_all_codons2[0:]['tr_id1'].tolist()))
b.columns = ['tr_id1']
b = b.merge(meta[['tr_id1', 'gene']])

# exclude genes 
b = b[~b['gene'].isin(transl_with_all_codons2['gene_ext'].tolist())]

b.gene.nunique(), b.tr_id1.nunique()

(3485, 8485)

In [ ]:
# number of genes with non-truncated CDS and with theretical ext length >= 20 codons is 11813 (28336 transcripts)
# number of genes with all of that exclusing genes predicted with Trips-viz (all 10k): 2819 (5686 transcripts) 

In [28]:
LIST_OF_FILT = b['tr_id1'].tolist()

In [39]:
out_counts = {'amb':{}, 'unamb':{}, 'all':{}}

for i, tr_id in enumerate(LIST_OF_FILT):   
    
    if i % 100 == 0:
        print ('transcript %s is processed out of 8485' % i)
    # end of transcript = its length
    tr_length = len(metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].transcript_seq)
    tr_seq = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].transcript_seq
    gene = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].gene
            
    if tr_id in list_of_unamb_tr:    
            
        tr_path_unamb = '../../data/counts_ext/download/N_term_unamb/%s_counts.txt' % tr_id
        df_unamb = pd.read_csv(tr_path_unamb, sep='\t')
        d_unamb = dict(zip(df_unamb['pos'].tolist(), df_unamb['Total'].tolist()))
            
        for i in range(0, tr_length):
            if i not in d_unamb:
                d_unamb[i] = 0
                
        out_counts['unamb'][tr_id] = d_unamb 
        
    else: # if tr_id is not in amb list
        continue
        #print ('%s is not in unamb list' % tr_id)
           
            
# save it to tmp_res            
with open('RiboSET_3485_genes.pickle', 'wb') as handle:
    pickle.dump(out_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

transcript 0 is processed out of 8485
transcript 100 is processed out of 8485
transcript 200 is processed out of 8485
transcript 300 is processed out of 8485
transcript 400 is processed out of 8485
transcript 500 is processed out of 8485
transcript 600 is processed out of 8485
transcript 700 is processed out of 8485
transcript 800 is processed out of 8485
transcript 900 is processed out of 8485
transcript 1000 is processed out of 8485
transcript 1100 is processed out of 8485
transcript 1200 is processed out of 8485
transcript 1300 is processed out of 8485
transcript 1400 is processed out of 8485
transcript 1500 is processed out of 8485
transcript 1600 is processed out of 8485
transcript 1700 is processed out of 8485
transcript 1800 is processed out of 8485
transcript 1900 is processed out of 8485
transcript 2000 is processed out of 8485
transcript 2100 is processed out of 8485
transcript 2200 is processed out of 8485
transcript 2300 is processed out of 8485
transcript 2400 is processed

In [40]:
!mkdir profiles

mkdir: cannot create directory ‘profiles’: File exists


In [41]:
out_df_frames_un = {}

for i, tr_id in enumerate(list(out_counts['unamb'].keys())):
    
    if i % 1000 == 0:
        print (i)
        
    d = out_counts['unamb'][tr_id]
    # d = dict with positioons and counts 
    frames = {'Frame_1': {}, 'Frame_2': {}, 'Frame_3': {} }
    for pos, counts in d.items():
        if pos % 3 == 2:
            frames['Frame_1'][pos] = counts
        elif pos % 3 == 0:
            frames['Frame_2'][pos] = counts
        else:
            frames['Frame_3'][pos] = counts
    # make df 
    li1 = []
    li2 = []
    li3 = []
    
    # end of transcript = its length
    tr_length = len(metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].transcript_seq)
    tr_seq = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].transcript_seq
    gene = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(tr_id)].iloc[0].gene
    
    for i in range(1, tr_length+1):
        if i in frames['Frame_1']:
            li1.append([i, int(frames['Frame_1'][i])])
        else:
            li1.append([i, int(0)])
    
    df1 = pd.DataFrame(li1, columns = ['Position', 'Frame 1']).sort_values(by='Position')
    
    for i in range(1, tr_length+1):
        if i in frames['Frame_2']:
            li2.append([i, int(frames['Frame_2'][i])])
        else:
            li2.append([i, int(0)])
    
    df2 = pd.DataFrame(li2, columns = ['Position', 'Frame 2']).sort_values(by='Position')
    
    for i in range(1, tr_length+1):
        if i in frames['Frame_3']:
            li3.append([i, int(frames['Frame_3'][i])])
        else:
            li3.append([i, int(0)])
    
    df3 = pd.DataFrame(li3, columns = ['Position', 'Frame 3']).sort_values(by='Position')
    
    df_f = df1.merge(df2, on='Position', how='inner').merge(df3, on='Position', how='inner')
    
    df_f['Sequence'] = list(tr_seq)
    
    df_f['Position'] = df_f['Position'] -1 #-3
    
    df_f['rna_seq'] = None
    
    df_f[['Position', 'Sequence', 'Frame 1', 'Frame 2', 'Frame 3', 'rna_seq']].to_csv('profiles/%s_%s_unamb.csv' % (gene, tr_id), index=False)

            
    out_df_frames_un[tr_id] = df_f
    
# save it out_df_frames_un 

with open('RiboSET_3485_genes_frames.pickle', 'wb') as handle:
    pickle.dump(out_df_frames_un, handle, protocol=pickle.HIGHEST_PROTOCOL)

0
1000
2000
3000
4000
5000
6000
7000
8000


In [42]:
li = []

for i,el in enumerate(list(out_df_frames_un.keys())):
    
    if i % 1000 == 0:
        print (i)
    N_term_start = meta[meta['tr_id'].str.contains(el)].iloc[0].N_term_start
    N_term_end = meta[meta['tr_id'].str.contains(el)].iloc[0].N_term_end
    
    cds_start = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(el)].iloc[0].cds_start_pos
    cds_stop = metadata_pc_g25[metadata_pc_g25['tr_id'].str.contains(el)].iloc[0].cds_stop_pos 

    
    # get CDS frame and CDS array 
    if cds_start % 3 == 0:
        frame = 'Frame 1'        
    elif cds_start % 3 == 1:
        frame = 'Frame 2'        
    else:
        frame = 'Frame 3'
        
    # extract counts for that frame 
    counts = out_df_frames_un[el][['Position', frame]]
    
    # CDS_array and CDS_coverage per codon and nonzero pos fraction 
    CDS_array = counts[cds_start:cds_stop][frame].tolist() 
    CDS_coverage = np.sum(CDS_array) / ((cds_stop - cds_start)//3)
    
    # number of non-zero pos / len of CDS in codons 
    CDS_nonzero_fraction = 3*np.count_nonzero(CDS_array) / (len(CDS_array)) 
    
    #if frame == 'Frame 1':
    #    CDS_array1 = [CDS_array[i] for i in range(0, len(CDS_array), 3)]
    #elif frame == 'Frame 2':
    CDS_array1 = [CDS_array[i] for i in range(1, len(CDS_array), 3)]
    #else:
    #    CDS_array1 = [CDS_array[i] for i in range(2, len(CDS_array), 3)]
        
    
    theor_ext_array = counts[N_term_start:N_term_end][frame].tolist()
    NTE_coverage = np.sum(theor_ext_array) / ((N_term_end - N_term_start) // 3)
    NTE_nonzero_fraction = 3*np.count_nonzero(theor_ext_array) / (len(theor_ext_array)) 
    
    
    CDS_std = np.std(CDS_array1)
    
    Z_score = (NTE_coverage - CDS_coverage) / CDS_std
    
    
    
    li.append([el, Z_score, 
               CDS_coverage, NTE_coverage, 
               CDS_nonzero_fraction, NTE_nonzero_fraction,
              ])
    
    
df = pd.DataFrame(li, columns = ['tr_id', 'Z_score', 
               'CDS_coverage', 'NTE_coverage', 
               'CDS_nonzero_fraction', 'NTE_nonzero_fraction'])


df.to_csv('3485_genes_df_coverages.txt', sep='\t', index=False)

0


/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in double_scalars
/home/alla/.local/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in double_scalars


1000
2000
3000
4000
5000
6000
7000
8000


In [43]:
subset = df[(df['NTE_nonzero_fraction'] < 0.0005) & 
               (df['NTE_coverage'] < 0.005) & 
               (df['CDS_coverage'] > 0.1) & 
               (df['CDS_nonzero_fraction'] > 0.3)].sort_values(by='CDS_coverage', ascending=False)

In [44]:
subset

,tr_id,Z_score,CDS_coverage,NTE_coverage,CDS_nonzero_fraction,NTE_nonzero_fraction
6019,ENST00000264203,-0.857147,543.402299,0.0,0.873563,0.0
1008,ENST00000276682,-1.039719,435.678474,0.0,0.956403,0.0
5076,ENST00000520813,-0.838518,286.958763,0.0,0.811644,0.0
3826,ENST00000409752,-0.692804,262.121053,0.0,0.878947,0.0
7609,ENST00000522794,-0.945111,219.881188,0.0,0.968699,0.0
...,...,...,...,...,...,...
4153,ENST00000615889,-0.616316,0.786517,0.0,0.393258,0.0
939,ENST00000589327,-0.616316,0.786517,0.0,0.393258,0.0
6540,ENST00000572449,-0.375367,0.740668,0.0,0.324165,0.0
1503,ENST00000442111,-0.400687,0.651297,0.0,0.314121,0.0
